In [11]:
from music21 import converter, instrument, note, chord, stream
import glob
import os
import numpy as np 
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Embedding
from keras.layers import Conv2D
from keras.layers import TimeDistributed
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [2]:
notes = [ ]

path = "C:\\Users\\barve\\Documents\\Deep-Learning\\lmd_full\\1\\"

#read all the filenames
files=[i for i in os.listdir("C:\\Users\\barve\\Documents\\Deep-Learning\\lmd_full\\1\\") if i.endswith(".mid")]

for file in files[0:50]:
    #print(file)
    try:
        midi = converter.parse(path + file)
    except:
        continue
    notes_to_parse = None

    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        #print(element)
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

print(notes) 

['5.9.0', '5', '5', '5.9.0', '5', '5', '5.9.0', '7.9.2', 'G3', '0', 'G3', 'C3', '0', '0.2.7', '7.0', '7.0', 'G3', '0', '9.2', 'A3', '2', 'A3', 'D3', '2', 'D4', '4.9', '9.2', '9.2', '9.2', 'F3', '10', 'F3', 'B-2', '5.10', 'B-3', 'C4', 'F3', 'F3', 'B-3', '5.10', '2.7', 'F3', 'A3', '0.5', 'F3', 'F3', '0.5', 'F3', 'A3', 'F3', '0.5', 'F3', '5.9.0', '5.9', '2.7', 'G3', '0', 'G3', 'C3', '7.0', 'C4', '2.7', 'G3', 'C3', 'G3', '0', '7.0', 'C3', '9.2', 'D3', 'A3', '2', 'A3', 'D3', '9.2', 'D4', '4.9', 'A3', '2', 'A3', '2', '9.2', 'F3', 'B-3', 'B-2', '5.10', 'F3', '10', '5.10', 'C4', '5.10', 'F3', '10', '5.10', '5.10', '2.7', '5.9', '0.5', '5', '5', '5.9', '0.5', '5', '5.9', '0.5', '5.9', '0.5', '2.7', 'A2', '7.0', 'C3', '7.0', '0', 'C4', '0.2.7', '7.0', '7.0', 'C3', '7.0', 'C3', '9.2', 'D3', '9.2', 'D3', '9.2', '2', 'D4', '2.4.9', '9.2', '9.2', 'D3', '9.2', 'D3', '5.10', 'B-2', '5.10', '10', 'B-3', '10.0.5', '5.10', '5.10', 'B-2', '5.10', 'B-2', '2.7', 'A2', '5.9', '0.5', '5', '5', '5.9', '0.5', '

In [3]:
sequence_length = 30

p_names = sorted(set(item for item in notes))

print(len(p_names))
#print(p_names)

note_int = dict()

#converting note to integers 
for number,note in enumerate(p_names):
    note_int[note] = number
    
print(len(note_int)) 
print(note_int) 

347
347
{'0': 0, '0.1': 1, '0.1.6': 2, '0.2': 3, '0.2.4': 4, '0.2.4.5': 5, '0.2.4.6.7.9': 6, '0.2.4.7': 7, '0.2.4.7.9': 8, '0.2.5': 9, '0.2.5.7': 10, '0.2.6': 11, '0.2.7': 12, '0.3': 13, '0.3.7': 14, '0.4': 15, '0.4.6': 16, '0.4.6.7': 17, '0.4.7': 18, '0.4.8': 19, '0.5': 20, '0.5.6': 21, '0.6': 22, '1': 23, '1.2': 24, '1.2.4': 25, '1.2.6': 26, '1.3': 27, '1.3.4': 28, '1.3.4.6': 29, '1.3.5': 30, '1.3.6': 31, '1.3.6.7': 32, '1.3.6.8': 33, '1.3.8': 34, '1.4': 35, '1.4.5': 36, '1.4.5.8': 37, '1.4.6': 38, '1.4.6.9': 39, '1.4.7': 40, '1.4.7.9': 41, '1.4.8': 42, '1.5': 43, '1.5.7': 44, '1.5.9': 45, '1.6': 46, '1.6.7': 47, '1.7': 48, '10': 49, '10.0': 50, '10.0.3': 51, '10.0.4': 52, '10.0.5': 53, '10.1': 54, '10.1.2': 55, '10.1.3.5': 56, '10.1.4': 57, '10.1.4.6': 58, '10.1.4.6.7': 59, '10.11': 60, '10.11.1': 61, '10.11.1.4.6': 62, '10.11.4': 63, '10.2': 64, '10.2.4': 65, '10.2.5': 66, '10.3': 67, '10.3.4': 68, '11': 69, '11.0.2.4.6.7': 70, '11.0.2.4.7': 71, '11.0.3.7': 72, '11.0.4.5.6': 73, '1

In [4]:
system_input = []
system_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    system_input.append([note_int[char] for char in sequence_in])
    system_output.append(note_int[sequence_out])
    
print(system_input[0])

[185, 172, 172, 185, 172, 172, 185, 230, 342, 0, 342, 296, 0, 12, 214, 214, 342, 0, 268, 272, 99, 272, 303, 99, 304, 170, 268, 268, 268, 329]


In [5]:
patterns = len(system_input)
# reshape the input into a format compatible with LSTM layers
system_input = np.reshape(system_input, (patterns, sequence_length, 1))
#print(system_input[0])
print(system_input.shape)
# normalize input
n_vocab = len(note_int)
n_system_input = system_input / float(n_vocab)
system_output = np_utils.to_categorical(system_output)

print(len(system_input))
print(len(system_output))
#print(system_input)

(51851, 30, 1)
51851
51851


In [6]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(system_input.shape[1], system_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=False))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 256)           264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               1574912   
_________________________________________________________________
dense_1 (Dense)              (None, 347)               178011    
_________________________________________________________________
activation_1 (Activation)    (None, 347)               0         
Total params: 2,017,115
Trainable params: 2,017,115
Non-trainable params: 0
_________________________________________________________________


In [7]:
filepath = "model-4-weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(n_system_input, system_output, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
51851/51851 [==============================] - 480s 9ms/step - loss: 4.3646
Epoch 2/50
51851/51851 [==============================] - 480s 9ms/step - loss: 4.0657
Epoch 3/50
51851/51851 [==============================] - 480s 9ms/step - loss: 3.6071
Epoch 4/50
51851/51851 [==============================] - 481s 9ms/step - loss: 3.2474
Epoch 5/50
51851/51851 [==============================] - 450s 9ms/step - loss: 2.9439
Epoch 6/50
51851/51851 [==============================] - 448s 9ms/step - loss: 2.6843
Epoch 7/50
51851/51851 [==============================] - 447s 9ms/step - loss: 2.4599
Epoch 8/50
51851/51851 [==============================] - 478s 9ms/step - loss: 2.2587
Epoch 9/50
51851/51851 [==============================] - 448s 9ms/step - loss: 2.0799
Epoch 10/50
51851/51851 [==============================] - 408s 8ms/step - loss: 1.9168
Epoch 11/50
51851/51851 [==============================] - 431s 8ms/step - loss: 1.7676
Epoch 12/50
51851/51851 [================

In [8]:
model.load_weights('model-4-weights-improvement-50-0.2675-bigger.hdf5')

In [9]:
start = np.random.randint(0, len(system_input)-1)
int_note = dict((number, note) for number, note in enumerate(p_names))
pattern = system_input[start]
prediction_output = []
print(start)
#print(pattern)
# generate 500 notes
for note_index in range(20):
    #prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    #prediction_input = prediction_input / float(n_vocab)
    #print(prediction_input.shape)
    pattern = np.reshape(pattern, (1,len(pattern),1))
    prediction = model.predict(pattern)
    index = np.argmax(prediction)
    result = int_note[index]
    prediction_output.append(result)
    #print(index)
    print(pattern)
    print(prediction_output)
    pattern = np.insert(pattern[0],len(pattern[0]),index)
    pattern = pattern[1:]

40272
[[[272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [ 35]
  [273]
  [272]
  [271]
  [272]
  [271]
  [290]
  [272]
  [271]
  [317]
  [290]
  [273]
  [272]
  [271]
  [272]
  [271]]]
['B3']
[[[271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [ 35]
  [273]
  [272]
  [271]
  [272]
  [271]
  [290]
  [272]
  [271]
  [317]
  [290]
  [273]
  [272]
  [271]
  [272]
  [271]
  [284]]]
['B3', 'B3']
[[[272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [ 35]
  [273]
  [272]
  [271]
  [272]
  [271]
  [290]
  [272]
  [271]
  [317]
  [290]
  [273]
  [272]
  [271]
  [272]
  [271]
  [284]
  [284]]]
['B3', 'B3', 'E2']
[[[271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [272]
  [271]
  [ 35]
  [273]
  [272]
  [271]
  [272]
  [271]
  [290]
  [272]
  [271]
  [317]
  [290]
  [273]
  [272]
  [271]
  [272]
  [271]
  [284]
  [28

In [12]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5
print(output_notes)

[<music21.note.Note B>, <music21.note.Note B>, <music21.note.Note E>, <music21.note.Note B>, <music21.note.Note E>, <music21.note.Note D>, <music21.note.Note E>, <music21.note.Note B>, <music21.note.Note B>, <music21.note.Note E>, <music21.note.Note B>, <music21.note.Note B>, <music21.note.Note B>, <music21.note.Note B>, <music21.note.Note E>, <music21.note.Note B>, <music21.note.Note B>, <music21.note.Note B>, <music21.note.Note B>, <music21.note.Note B>]


In [13]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='lstm_output_1.mid')

'lstm_output_1.mid'